In [1]:
import nltk
import numpy as np
import pandas as pd
from pandas import DataFrame
from keras.models import Model, Sequential, load_model
from keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dropout, Dense, Flatten
from keras.layers import TimeDistributed, Bidirectional, InputLayer, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
from keras.metrics import categorical_accuracy
from IPython.display import clear_output
from more_itertools import flatten, intersperse
import random

import os
import urllib.request

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2
from batcher import batch_from_generator
from train_data import load_conll2003, create_conll_encoded_shifted_generator
from mappings import get_all_mappings
from corpus import corpus_training_data_generator, create_all_corpus_train_pipeline
from padder import pad
from model import create_model, compile_model

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [38]:
DROPOUT = 0.2
TIME_SLICE_SIZE = 128
BATCH_SIZE = 512
SAMPLING_RATE = 1
PADDING = 0
UNKNOWN = 1
NUM_OF_UNITS = 250
WORDS_PER_BATCH = 100
EPOCHS=1
MODEL_SAVE_PATH = 'tc_model.h5'
LSTM_MODEL_SAVE_PATH = 'lstm_model.h5'

In [39]:
mapping, reverse_mapping, lower_mapping, lower_reverse_mapping = get_all_mappings()

In [40]:
# def create_model(shape, classes):
#     # First layer inputs must be 3D
#     # with shape (samples, timesteps, features)
#     model = Sequential()
#     model.add(InputLayer(input_shape=shape))
#     model.add(Bidirectional(LSTM(NUM_OF_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=DROPOUT)))
#     model.add(TimeDistributed(Dense(1000)))
#     model.add(TimeDistributed(Dense(classes)))
#     model.add(Activation('softmax'))
#     return model

In [41]:
model = create_model(len(mapping), NUM_OF_UNITS, DROPOUT)

In [42]:
model = compile_model(model)

In [33]:
if os.path.isfile(MODEL_SAVE_PATH): 
    model.load_weights(MODEL_SAVE_PATH)

In [34]:
tensor_board = TensorBoard(batch_size=BATCH_SIZE, write_graph=False, )

In [35]:
X, Y = create_all_corpus_train_pipeline(TIME_SLICE_SIZE)

In [ ]:

#g = corpus_training_data_generator('gutenberg',TIME_SLICE_SIZE, BATCH_SIZE, shift=False)
model.fit(X, Y,  verbose=2, validation_split=0.2, callbacks=[tensor_board], batch_size=200, epochs=100)

Train on 112519 samples, validate on 28130 samples
Epoch 1/100


In [53]:
test_sentence = pad("I want to go on holiday to France".lower(), TIME_SLICE_SIZE)
mapped_sentence = [lower_mapping[char] for char in test_sentence]
mapped_sentence = pad_sequences([mapped_sentence], maxlen=TIME_SLICE_SIZE, padding='pre')
mapped_sentence = to_categorical(mapped_sentence, len(mapping))

mapped_sentence = np.asarray(mapped_sentence)
#mapped_sentence = np.reshape(1, len(test_sentence))
predicted_result = model.predict_classes(mapped_sentence)
predicted_result = [reverse_mapping[i] for i in predicted_result[0]]
''.join(predicted_result) 

'I want to go on holiday to France                                                                                               '

In [54]:
model.save("big_lstm.h5")